In [1]:
##################################
#
# Initial implementation of linear logic recurrent neural network
#
# The architecture is a modified RNN, see the paper "Linear logic and recurrent neural networks".
# Our inputs are sequences of symbols taken from an alphabet of size num_classes. The length
# of the sequences is N. Our outputs are also sequences of length N from the same alphabet.
#
# An input symbol is encoded as a one_hot vector, so our inputs are sequences of one-hot vectors
# of size num_classes. Put another way, this is the dimension of the input space.
#
# The function to be learned is func_to_learn
#
# At present at each time step the RNN simply emits its hidden state. These outputs are
# then passed through a fully-connected layer (one by one) to generate the output symbol
# at each time step.

# TODO:
#
# Dropout
# Layer normalisation
# See "Recurrent neural networks in TensorFlow II"

# Global flags
num_classes = 2
batch_size = 100
state_size = 24
input_size = num_classes
N = 15 # length of sequences
PERCENT_EXAMPLES = 0.1 # number of training examples
epoch = 10

In [2]:
# The next three lines are recommend by TF
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import tensorflow as tf
import numpy as np
import collections
import six
import math

from random import shuffle
from tensorflow.python.ops import variable_scope as vs
from tensorflow.python.ops.rnn_cell import RNNCell
from tensorflow.python.ops import array_ops
from tensorflow.python.ops import init_ops
from tensorflow.python.ops.math_ops import sigmoid
from tensorflow.python.ops.math_ops import tanh

In [3]:
def church(n,A):
    if n < 2:
        raise ValueError("church is expecting an integer > 1")
    if n == 2:
        return(tf.matmul(A,A))
    if n > 2:
        return(tf.matmul(A, church(n-1,A)))
    
    # This should never be reached
    return A

In [4]:
# In tf.nn.rnn_cell there is a private function _linear that 
# we have modified here to be used in our LinearLogicRNNCell

# A 2D tensor of shape [X,Y] means a matrix with X rows and Y cols
# The row index here is interpreted as indexing into a batch.
    
class LinearLogicRNNCell(RNNCell):
    
    def __init__(self, num_units, input_size=None, activation=tanh):
        if input_size is not None:
            logging.warn("%s: The input_size parameter is deprecated." % self)
        self._num_units = num_units
        self._activation = activation
    
    @property
    def state_size(self):
        return self._num_units

    @property
    def output_size(self):
        return self._num_units

    def __call__(self, inputs, state, scope=None):
        # the scope business gives a namespace to our weight variable matrix names
        # e.g. "Program", "Matrix" and "Bias"
        with vs.variable_scope(scope or "Linear"): 
            # Our input is a pair of 2D tensors of shapes [B, input_size] and [B, state_size]
            # where B is the batch size.

            # array_ops.concat(1,args) is a Tensor of shape [B, input_size + state_size]
            # which is obtained by stacking the matrices together horizontally.
            args = [inputs,state]
            
            # every row of inputs (resp. state) has length input_size (resp state_size)
            # prog_weights = vs.get_variable("Program", [(input_size+state_size)**2])
            # f = lambda x: tf.mul(tf.reshape(tf.matmul(tf.transpose(x),x),[-1]),prog_weights)
            # h = array_ops.concat(1, args) # (I|S)
            # h2 = tf.expand_dims(h,1)
            # h5 = tf.map_fn(f, h2)  
                
            # Comment out the following line to not add higher monomials
            # args = args + [h5]
            
            # Calculate the total size of arguments on dimension 1.
            total_arg_size = 0
            shapes = [a.get_shape().as_list() for a in args]

            for shape in shapes:
                total_arg_size += shape[1]           
    
            # NOTE: the H and U that are referred to in the RNN update equation of
            # the paper are the block parts of the following matrix.
            # Also, everything is transposed here compared to the paper, since we
            # need rows to correspond to entries in the batch in TensorFlow. So for
            # example we actually compute hH + xU + B.
            
            Lambda = [2,3] # meaning church(2,-), church(3,-)
            l = len(Lambda)
            #k = 2
            
            HU = vs.get_variable("HU", [total_arg_size, state_size])
            B = vs.get_variable("B", [state_size], initializer=init_ops.constant_initializer(0.0))
            #P = vs.get_variable("P", [state_size,l])
            #C = vs.get_variable("C", [l])
            #Q = vs.get_variable("Q", [state_size,l])
            #D = vs.get_variable("D", [k])
            V = vs.get_variable("V", [input_size, state_size**2])
            #p = self._activation(tf.matmul(state,P) + C)
            #q = self._activation(tf.matmul(state,Q) + D)
            Vx = tf.matmul(inputs,V)
            
            # Each row of Vx has state_size**2 entries and is to be interpreted as a matrix
            Vx_matrices = tf.reshape(Vx,[-1,state_size,state_size])
            f = lambda A: church(2,A)
            G = tf.map_fn(f,Vx_matrices)
            S = tf.expand_dims(state,1)
            G = tf.batch_matmul(S,G)
            G = tf.reshape(G,[-1,state_size]) # in each batch, we get state * (Vx)^n
            # G is [lambda](Vx)(h)
            
            res = tf.matmul(array_ops.concat(1, args), HU)
            output = self._activation(res + G + B)
        return output, output
        # note that as currently written the RNN emits its internal state at each time step

In [5]:
# The function from sequences to sequences that we will try to learn

def f_ident(seq):
    return seq

# reverse is very hard for the current configuration
def f_reverse(seq):
    t = [0]*len(seq)
    for j in range(len(seq)):
        t[len(seq)-j-1] = seq[j]
    return t

def f_swap01(seq):
    t = []
    for j in range(len(seq)):
        if seq[j] == 0:
            t.append(0)
        else:
            t.append(1)
    return t

def f1(seq):
    t = []
    for j in range(len(seq)):
        if seq[j] == 1 and j > 0 and seq[j-1] == 1:
            if j > 3:
                t.append(seq[j-2])
            else:
                t.append(0)
        else:
            t.append(1)
    return t

func_to_learn = f_reverse

In [6]:
# Create a shuffled list of all binary sequences of length N
s = '{0:0' + str(N) + 'b}'
seq_input = [s.format(i) for i in range(2**N)]
shuffle(seq_input)
seq_input = [map(int,i) for i in seq_input]
ti = []
for i in seq_input:
    temp_list = []
    for j in i:
        temp_list.append(j)
    ti.append(temp_list)
seq_input = ti

print("Number of sequences: " + str(len(seq_input)))
print(seq_input[0])
# A typical element of seq_input at this point will be an array like
# array([[1],[0],[1],[1],[0]])

one_hots = []
for i in range(num_classes):
    a = [0.0]*num_classes
    a[i] = 1.0
    one_hots.append(np.array(a))

seq_input_onehot = []
for i in seq_input:
    temp_list = []
    for j in i:
        temp_list.append(one_hots[j])
    seq_input_onehot.append(np.array(temp_list))

print(seq_input_onehot[0])

Number of sequences: 32768
[1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 0, 0]
[[ 0.  1.]
 [ 1.  0.]
 [ 1.  0.]
 [ 0.  1.]
 [ 1.  0.]
 [ 0.  1.]
 [ 0.  1.]
 [ 0.  1.]
 [ 1.  0.]
 [ 0.  1.]
 [ 1.  0.]
 [ 0.  1.]
 [ 0.  1.]
 [ 1.  0.]
 [ 1.  0.]]


In [7]:
# Training output
seq_output = []

# Swaps 0s for 1s in each sequence
for i in seq_input:
    seq_output.append(func_to_learn(i))
                    
print(seq_output[0])

seq_output_onehot = []
for i in seq_output:
    temp_list = []
    for j in i:
        temp_list.append(one_hots[j])
    seq_output_onehot.append(np.array(temp_list))

print(seq_output_onehot[0])

[0, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1]
[[ 1.  0.]
 [ 1.  0.]
 [ 0.  1.]
 [ 0.  1.]
 [ 1.  0.]
 [ 0.  1.]
 [ 1.  0.]
 [ 0.  1.]
 [ 0.  1.]
 [ 0.  1.]
 [ 1.  0.]
 [ 0.  1.]
 [ 1.  0.]
 [ 1.  0.]
 [ 0.  1.]]


In [8]:
NUM_EXAMPLES = int(PERCENT_EXAMPLES * len(seq_input))
print("Number of training examples: " + str(NUM_EXAMPLES) + "/" + str(len(seq_input)))

test_input = seq_input_onehot[NUM_EXAMPLES:3*NUM_EXAMPLES]
test_output = seq_output_onehot[NUM_EXAMPLES:3*NUM_EXAMPLES]
train_input = seq_input_onehot[:NUM_EXAMPLES]
train_output = seq_output_onehot[:NUM_EXAMPLES]

print("")
print("The first one-hot encoded digit of the first three output sequences")
print(test_output[0][0])
print(test_output[1][0])
print(test_output[2][0])

Number of training examples: 3276/32768

The first one-hot encoded digit of the first three output sequences
[ 0.  1.]
[ 1.  0.]
[ 1.  0.]


In [9]:
# Definition of the model

# inputs, we create N of them, each of shape [None,input_size], one for
# each position in the sequence
inputs = [tf.placeholder(tf.float32, [None,input_size]) for _ in range(N)]
targets = [tf.placeholder(tf.float32, [None,input_size]) for _ in range(N)]

# We use tf.nn.rnn rather than dynamic_rnn because there appears to
# be a problem with tf.map_fn and the latter, at least in 0.10
# state_size is the number of hidden neurons in each layer
cell = LinearLogicRNNCell(state_size)

# tf.nn.rnn returns a pair, the first is a list of the
# outputs from each step, the second is the final internal state.
rnn_outputs, last_state = tf.nn.rnn(cell,inputs,dtype=tf.float32)

# Final fully connected layer
E = tf.Variable(tf.truncated_normal([state_size,input_size]))
F = tf.Variable(tf.constant(0.1, shape=[input_size]))

# prediction is a length N list of tensors of shape [None,input_size], where
# the jth row of prediction[d] is, for the jth input sequence in the batch,
# the probability distribution over symbols for the output symbol in position d.
logits = [tf.matmul(rnn_output, E) + F for rnn_output in rnn_outputs]
prediction = [tf.nn.softmax(logit) for logit in logits] 
ce = [tf.reduce_sum(targets[i] * tf.log(prediction[i])) for i in range(N)]

cross_entropy = -tf.add_n(ce)
optimizer = tf.train.AdamOptimizer()
minimize = optimizer.minimize(cross_entropy)

mistakes = [tf.not_equal(tf.argmax(targets[i], 1), tf.argmax(prediction[i], 1)) for i in range(N)]
errors = [tf.reduce_mean(tf.cast(m, tf.float32)) for m in mistakes]

In [10]:
# Initialise the model
init_op = tf.initialize_all_variables()
sess = tf.Session()
sess.run(init_op)

In [11]:
# Display the errors before training
feed_dict = {}
for d in range(N):
    in_node = inputs[d]
    out_node = targets[d]
    
    ti = []
    to = []
    for k in range(len(test_input)):
        ti.append(test_input[k][d]) # A vector giving the one-hot encoding of the dth symbol in the kth sequence
        to.append(test_output[k][d])
    feed_dict[in_node] = np.array(ti)
    feed_dict[out_node] = np.array(to)

# The first three digits of this should match the printout for the
# first three test output sequences given earlier
print(sess.run(tf.argmax(targets[0],1),feed_dict))
print(sess.run(tf.argmax(prediction[0],1),feed_dict))
print(sess.run(tf.not_equal(tf.argmax(targets[0], 1), tf.argmax(prediction[0], 1)),feed_dict))

print("")
print("The mean of the errors in each digit for the test set:")
incorrects = sess.run(errors, feed_dict)
print(incorrects)

[1 0 0 ..., 0 0 0]
[0 0 0 ..., 0 0 0]
[ True False False ..., False False False]

The mean of the errors in each digit for the test set:
[0.49420023, 0.51037854, 0.49404761, 0.50213677, 0.4957265, 0.49343711, 0.497558, 0.51648355, 0.53586692, 0.50427353, 0.49435288, 0.52854091, 0.49832112, 0.48809522, 0.50839436]


In [12]:
# Training
no_of_batches = int(len(train_input)/batch_size)
print("Number of batches: " + str(no_of_batches))

# An annoying thing here is that we cannot use a list as a key in a 
# dictionary. The workaround we found on StackOverflow here:
# http://stackoverflow.com/questions/33684657/issue-feeding-a-list-into-feed-dict-in-tensorflow)

# epoch is a global var
for i in range(epoch):
    ptr = 0
    for j in range(no_of_batches):
        inp = train_input[ptr:ptr+batch_size]
        out = train_output[ptr:ptr+batch_size]
        ptr += batch_size
        
        feed_dict = {}
        for d in range(N):
            in_node = inputs[d]
            out_node = targets[d]
            
            # inp has dimensions [batch_size, N, num_classes] and we want to extract
            # the 2D Tensor of shape [batch_size, num_classes] obtained by setting the
            # second coordinate to d
            ti = []
            to = []
            for k in range(batch_size):
                ti.append(inp[k][d])
                to.append(out[k][d])

            feed_dict[in_node] = np.array(ti)
            feed_dict[out_node] = np.array(to)
            
        sess.run(minimize, feed_dict)
    print("Epoch - " + str(i+1))

Number of batches: 32
Epoch - 1
Epoch - 2
Epoch - 3
Epoch - 4
Epoch - 5
Epoch - 6
Epoch - 7
Epoch - 8
Epoch - 9
Epoch - 10


In [13]:
# Display the errors after training
feed_dict = {}
for d in range(N):
    in_node = inputs[d]
    out_node = targets[d]
    
    ti = []
    to = []
    for k in range(len(test_input)):
        ti.append(test_input[k][d]) # A vector giving the one-hot encoding of the dth symbol in the kth sequence
        to.append(test_output[k][d])
    feed_dict[in_node] = np.array(ti)
    feed_dict[out_node] = np.array(to)

# The first three digits of this should match the printout for the
# first three test output sequences given earlier
data = sess.run([tf.argmax(targets[0],1),
                 tf.argmax(prediction[0],1),
                 cross_entropy],feed_dict)

print("First digits of test outputs (actual)")
print(data[0])
print("First digits of test outputs (predicted)")
print(data[1])
print("Cross-entropy: " + str(data[2]))

# print the mean of the errors in each digit for the test set.
incorrects = sess.run(errors, feed_dict)
print(incorrects)

sess.close()

First digits of test outputs (actual)
[1 0 0 ..., 0 0 0]
First digits of test outputs (predicted)
[0 0 0 ..., 0 0 0]
Cross-entropy: 117365.0
[0.49420023, 0.51037854, 0.49404761, 0.49862638, 0.50274724, 0.49420023, 0.49801588, 0.50885224, 0.50595236, 0.4781746, 0.49252138, 0.51663613, 0.50656289, 0.50061053, 0.49328449]
